<a href="https://colab.research.google.com/github/morikun1030/BURI/blob/master/edinet_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EDINETからdocID（書類管理番号）を取得する**

参考:[Zenn](https://zenn.dev/robes/articles/f6dfcc5cfbbdb6)

# [EDINETからdocID（書類管理番号）を取得する（EDINET API version2対応）](https://zenn.dev/robes/articles/f6dfcc5cfbbdb6)

In [1]:
!pip install google-colab

In [2]:
!pip install --upgrade google-colab

In [3]:
import os
from google.colab import userdata
from google.colab import drive
import requests
import datetime
import pandas as pd
import time
import zipfile
import warnings
from tqdm.notebook import tqdm
import io

In [5]:
# Google Driveのマウント
drive.mount('/content/drive')

# 基本ディレクトリの設定
base_dir = "/content/drive/MyDrive/directory/"

# 警告を特定のものに限定する
warnings.filterwarnings('ignore', category=DeprecationWarning)

# シークレットキーの取得
edinet_key = userdata.get('EDINET')

# シークレットキーが存在しない場合のエラーハンドリング
if not edinet_key:
    raise ValueError("EDINET secret key is not set in userdata.")

# 環境変数の設定
os.environ['EDINET'] = edinet_key

# 確認のために環境変数をプリント
print("EDINET key:", os.environ.get('EDINET'))

class GetDocid:
    #1 コンストラクタ・日付リストの作成
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date
        self.day_list = self._create_day_list()

    def _create_day_list(self):
        day_list = []
        period = (self.end_date - self.start_date).days
        for d in range(period + 1):
            day = self.start_date + datetime.timedelta(days=d)
            day_list.append(day)
        return day_list

    #2 レポートリストの作成
    def create_report_list(self):
        report_list = []
        for day in tqdm(self.day_list, desc="Downloading reports"):
            url = "https://api.edinet-fsa.go.jp/api/v2/documents.json"
            params = {"date": day, "type": 2, "Subscription-Key": edinet_key}
            try:
                res = requests.get(url, params=params)
                res.raise_for_status()
                json_data = res.json()
                time.sleep(3)
            except requests.RequestException as e:
                print(f"Request failed: {e}")
                continue
            for result in json_data.get("results", []):
                if result["ordinanceCode"] == "010" and result["formCode"] == "030000":
                    report_list.append({
                        '会社名': result["filerName"],
                        '書類名': result["docDescription"],
                        'docID': result["docID"],
                        '証券コード': result["secCode"],
                        'ＥＤＩＮＥＴコード': result["edinetCode"],
                        '決算期': result["periodEnd"],
                        '提出日': day
                    })
        return report_list

    #データフレームの作成と保存
    def create_docid_df(self, base_dir):
        # ファイルパスを動的に設定
        zip_path = os.path.join(base_dir, "Edinetcode.zip")
        extract_dir = os.path.join(base_dir, "Edinetcode")
        df_info_path = os.path.join(extract_dir, "EdinetcodeDlInfo.csv")
        output_path = os.path.join(base_dir, "csv", "edinet_df.csv")
        # 出力ディレクトリを作成
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # edinetcodelistを取得する
        zip_url = "https://disclosure2dl.edinet-fsa.go.jp/searchdocument/codelist/Edinetcode.zip"

        # ZIPファイルをダウンロード
        try:
            response = requests.get(zip_url)
            response.raise_for_status()
            os.makedirs(os.path.dirname(zip_path), exist_ok=True)
            with open(zip_path, "wb") as f:
                f.write(response.content)
        except requests.exceptions.RequestException as e:
            print(f"Failed to download ZIP file: {e}")
            return None

        # zipファイルのダウンロードと展開
        try:
            with zipfile.ZipFile(zip_path) as zip_f:
                zip_f.extractall(extract_dir)
        except Exception as e:
            print(f"Failed to unzip file: {e}")
            return None

        df_info = pd.read_csv(df_info_path, encoding="cp932", skiprows=[0])
        df_report = pd.DataFrame(self.create_report_list())
        df_info = df_info[["ＥＤＩＮＥＴコード", "提出者業種"]]
        merged_df = pd.merge(df_report, df_info, how="inner", on="ＥＤＩＮＥＴコード")
        merged_df.to_csv(output_path, index=False)

        if merged_df is None:
            return None
        else:
            return merged_df

#書類提出期間の指定
start_date = datetime.date(2024, 3, 1)
end_date = datetime.date(2024, 3, 10)
gd = GetDocid(start_date, end_date)
edinet_df = gd.create_docid_df(base_dir)

#クラスのインスタンス化
gd = GetDocid(start_date,end_date)

#docIDのデータフレーム取得
edinet_df = gd.create_docid_df(base_dir)

#docid_listの作成
docid_list = edinet_df["docID"].tolist()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
EDINET key: 6529a982d1144470be1d2fc0711ca023


In [6]:
#print(edinet_df)

# **[EDINETから有価証券報告書をXBRLファイル形式で取得し、テキストデータを抽出する](https://zenn.dev/robes/articles/b7be539b65ee9d)**

In [7]:
from bs4 import BeautifulSoup
import zipfile
import io
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time
import glob
import re

In [8]:
from google.colab import userdata
userdata.get("EDINET")

'6529a982d1144470be1d2fc0711ca023'

In [9]:
edinet_key = os.environ.get('EDINET')
if not edinet_key:
    raise ValueError("EDINET environment variable is not set.")

In [10]:
class GetReportFromEdinet:

    def __init__(self,docid_list):
        self.docid_list = docid_list

    def get_xbrl_file(self):
        for docid in tqdm(self.docid_list):
            # 書類取得APIのエンドポイント
            url = "https://api.edinet-fsa.go.jp/api/v2/documents/" + docid
            print(url)
            time.sleep(5)


            #書類取得APIのパラメータ
            params ={"type":1,"Subscription-Key":edinet_api_key}
            res = requests.get(url,params=params, verify=False)

            #ファイルへの出力
            print(res.status_code)
            if res.status_code == 200:
            # レスポンスからZIPファイルを読み込む
                with zipfile.ZipFile(io.BytesIO(res.content)) as z:
                # ZIPファイル内のすべてのファイルをループ処理
                    for file in z.namelist():
                        if file.startswith("XBRL/PublicDoc/") and file.endswith(".xbrl"):
                        # .xbrlファイルを見つけたら、それをディスクに書き込む
                            z.extract(file, path=f'/content/{docid}/')

In [11]:
grfe = GetReportFromEdinet(docid_list)

In [22]:
grfe.get_xbrl_file()

  0%|          | 0/2 [00:00<?, ?it/s]

https://api.edinet-fsa.go.jp/api/v2/documents/S100SZWZ


NameError: name 'edinet_api_key' is not defined

**テキストデータを抽出する**
ここでは、テキストデータとして以下の非財務情報を抽出していきます。

*   経営方針、経営環境及び対処すべき課題等
*   事業等のリスク
*   従業員の状況
*   サステナビリティに関する考え方及び取組

In [23]:
#xbrlパーサのインストール
!pip install edinet_xbrl

In [14]:
#xbrlパーサのインポート
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser

In [15]:
parser = EdinetXbrlParser()

**クラスの定義**

In [16]:
class GetTextFromXBRL:

    def __init__(self, keys, context_ref, docid_list):
        self.keys = keys
        self.context_ref = context_ref
        self.docid_list = docid_list

    def parse_xbrl(self, xbrl_path):
        try:
            return parser.parse_file(xbrl_path), None
        except Exception as e:
            return None, str(e)

    def extract_data_from_xbrl(self, edinet_xbrl_object, key):
        if not edinet_xbrl_object:
            return None
        data = edinet_xbrl_object.get_data_by_context_ref(key, self.context_ref)
        if data:
            value = data.get_value()
            if value:
                return BeautifulSoup(value, 'html.parser').get_text(strip=True)
        return None

     #EDINETコードの取得
    def extract_dynamic_code(self,xbrl_path):
        pattern = r'jpcrp030000-asr-001_E(\d{5})'
        match = re.search(pattern, xbrl_path)
        if match:
            return "E" + match.group(1)
        else:
            return None

    def get_text_data_generic(self, include_sustaina=True):
        columns = ["docID"] + list(self.keys.keys())
        if include_sustaina:
            columns.append("サステナビリティ方針")
        extracted_data_list = []

        for _docid in tqdm(self.docid_list):
            xbrl_path = f'/content/{_docid}/XBRL/PublicDoc/*.xbrl'
            xbrl_paths = glob.glob(xbrl_path)
            if xbrl_paths:
                xbrl_path = xbrl_paths[0]
                edinet_xbrl_object, error = self.parse_xbrl(xbrl_path)
                if error:
                    print(f"Error parsing XBRL for {_docid}: {error}")
                    continue

                row_data = [_docid]
                for key_text in self.keys.values():
                    extracted_text = self.extract_data_from_xbrl(edinet_xbrl_object, key_text)
                    row_data.append(extracted_text)

                if include_sustaina:
                    dynamic_code = self.extract_dynamic_code(xbrl_path)
                    key_text = f"jpcrp030000-asr_{dynamic_code}-000:DisclosureOfSustainabilityRelatedFinancialInformationTextBlock"
                    extracted_text = self.extract_data_from_xbrl(edinet_xbrl_object, key_text)
                    row_data.append(extracted_text)

                extracted_data_list.append(row_data)

        return pd.DataFrame(extracted_data_list, columns=columns)


**クラスの実行**
keys（辞書）・cotext_refの指定

In [17]:
keys = {
    "経営方針":"jpcrp_cor:BusinessPolicyBusinessEnvironmentIssuesToAddressEtcTextBlock", #経営方針、経営環境および対処すべき課題等
    "事業等のリスク":"jpcrp_cor:BusinessRisksTextBlock", #事業等のリスク
    "従業員の状況":"jpcrp_cor:InformationAboutEmployeesTextBlock", #従業員の状況
}

context_ref = "FilingDateInstant"

**クラスのインスタンス化**

In [18]:
gtfb = GetTextFromXBRL(keys,context_ref,docid_list)

**クラスメソッドの実行**

In [19]:
%%time
text_df = gtfb.get_text_data_generic()

  0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/xbrl/xbrl.py:26: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(fh, "lxml")


CPU times: user 1.76 s, sys: 80.2 ms, total: 1.84 s
Wall time: 2 s


**Edinetデータとの統合**

In [20]:
def merge_df(df1, df2, how="left"):
    #最初の2つのDataFrameを結合
    merged_df = pd.merge(df1, df2, on='docID', how=how)
    return merged_df

# merge_df()関数の戻り値を変数に代入
merged_df = merge_df(edinet_df, text_df)

output_dir = '/path/to/output/directory'
os.makedirs(output_dir, exist_ok=True)  # ディレクトリが存在しない場合は作成する
output_file = os.path.join(output_dir, 'merged_data.csv')
merged_df.to_csv(output_file, index=False, encoding='utf-8-sig')